In [ ]:
!pip install -r requirements.txt

In [ ]:
from collections.abc import Callable
from enum import Enum

from numpy import full, ndarray, zeros
from scipy.linalg import solve_banded

class Method(Enum):
    ALTERNATING_DIRECTION = 0,
    FRACTIONAL_STEP = 1


def splitting_method(alpha: ndarray, f_x_y_t: Callable,
    l: ndarray, n: tuple, t: float, k_upper: int,
    alpha1: float, beta1: float, alpha2: float, beta2: float,
    alpha3: float, beta3: float, alpha4: float, beta4: float,
    phi_1_x_t, phi_2_x_t, phi_3_y_t, phi_4_y_t, psi_x_y,
    method: Method, callback: Callable) -> ndarray:
    h, tau, sigma = l / n, t / k_upper, alpha * tau / (h * h)

    ai = np.full(n[0] + 1, sigma[0] / (2.0 if method == ALTERNATING_DIRECTION else 1.0))
    bi = np.full(n[0] + 1, -(1.0 if method == ALTERNATING_DIRECTION else 2.0) * sigma[0] - 1.0)
    ci = np.full(n[0] + 1, sigma[0] / (2.0 if method == ALTERNATING_DIRECTION else 1.0))
    aj = np.full(n[1] + 1, sigma[1] / (2.0 if method == ALTERNATING_DIRECTION else 1.0))
    bj = np.full(n[1] + 1, -(1.0 if method == ALTERNATING_DIRECTION else 2.0) * sigma[1] - 1.0)
    cj = np.full(n[1] + 1, sigma[1] / (2.0 if method == ALTERNATING_DIRECTION else 1.0))
    di, dj = np.zeros(n[0] + 1), np.zeros(n[1] + 1)

    ai[0] = c_i[-1] = a_j[0] = c_j[-1] = 0.0
    bi[0] = beta3 - alpha3 / h1
    ci[0] = alpha3 / h1
    ai[-1] = -alpha4 / h1
    bi[-1] = beta4 + alpha4 / h1
    bj[0] = beta1 - alpha1 / h2
    cj[0] = alpha1 / h2
    aj[-1] = -alpha2 / h2
    bj[-1] = beta2 + alpha2 / h2

    u, u_temp = np.zero((2, *n + 1))
    __initial_approximation(n[0], h[0], n[1], h[1], psi_x_y, u)
    for k in range(1, k_upper + 1):
        __step_x(f_x_y_t,
            n, h, tau, k, sigma, alpha1, beta1, alpha2, beta2,
            phi_1_x_t, phi_2_x_t, phi_3_y_t, phi_4_y_t, ai, bi, ci, di,
            method, u, u_temp)
        __step_y(f_x_y_t,
            n, h, tau, k, sigma_a, alpha3, beta3, alpha4, beta4,
            phi_1_x_t, phi_2_x_t, phi_3_y_t, phi_4_y_t, aj, bj, cj, dj,
            method, u_temp, u)
    return u


def __initial_approximation(n, h, psi_x_y, u0):
    for i in range(n[0] + 1):
        for j in range(n[1] + 1):
            u0[i, j] = psi_x_y(i * h[0], j * h[1])


def __step_x(f_x_y_t: Callable,
    n: tuple, h: tuple, tau: float, k: int, sigma: ndarray,
    alpha1: float, beta1: float, alpha2: float, beta2: float,
    phi_1_x_t, phi_2_x_t, phi_3_y_t, phi_4_y_t,
    ai: ndarray, bi: ndarray, ci: ndarray, di: ndarray,
    method: Method, u_k_minus_1: ndarray, u_k_minus_1_divides_2: ndarray):
    for j in range(1, n[1]):
        di[0], d_i[-1] = phi_3_y_t(j * h[1], (k - 0.5) * tau), phi_4_y_t(j * h[1], (k - 0.5) * tau)
        for i in range(1, n[0]):
            di[i] = (-u_k_minus_1[i, j]
                - tau / 2.0 * f_x_y_t(i * h[0], j * h[1], (k - 0.5) * tau))
            if method == ALTERNATING_DIRECTION:
                di[i] += (sigma[1] * u_k_minus_1[i, j]
                    - sigma[1] / 2.0 * u_k_minus_1[i, j - 1]
                    - sigma[1] / 2.0 * u_k_minus_1[i, j + 1])
        u_k_minus_1_divides_2[:, j] = thomas_algorithm(a_i, b_i, c_i, d_i)
    for i in range(0, n[0] + 1):
        u_k_minus_1_divides_2[i, 0] = (-alpha1 * u_k_minus_1_divides_2[i, 1]
                + h[1] * phi_1_x_t(i * h[0], (k - 0.5) * tau)) / (beta1 * h[1] - alpha1)
        u_k_minus_1_divides_2[i, -1] = (alpha2 * u_k_minus_1_divides_2[i, -2]
                + h[1] * phi_2_x_t(i * h[0], (k - 0.5) * tau)) / (beta2 * h[1] + alpha2)


def __step_y(f_x_y_t: Callable,
    n: tuple, h: tuple, tau: float, k: int, sigma: ndarray,
    alpha3: float, beta3: float, alpha4: float, beta4: float,
    phi_1_x_t, phi_2_x_t, phi_3_y_t, phi_4_y_t,
    ai: ndarray, bi: ndarray, ci: ndarray, di: ndarray,
    method: Method, u_k_minus_1_divides_2: ndarray, u_k: ndarray):
    for i in range(1, n[0]):
        d_j[0], d_j[-1] = phi_1_x_t(i * h[0], k * tau), phi_2_x_t(i * h[0], k * tau)
        for j in range(1, n[1]):
            d_j[j] = (-u_k_minus_1_divides_2[i, j]
                - tau / 2.0 * f_x_y_t(i * h[0], j * h[1], k * tau))
            if method == ALTERNATING_DIRECTION:
                d_j[j] += (sigma[0] * u_k_minus_1_divides_2[i, j]
                    - sigma[0] / 2.0 * u_k_minus_1_divides_2[i - 1, j]
                    - sigma[0] / 2.0 * u_k_minus_1_divides_2[i + 1, j])
        u_k[i, :] = thomas_algorithm(a_j, b_j, c_j, d_j)
    for j in range(0, n[1] + 1):
        u_k[0, j] = (-alpha3 * u_k[1, j]
            + h[0] * phi_3_y_t(a, b, mu, j * h[1], k * tau)) / (beta3 * h[0] - alpha3)
        u_k[-1, j] = (alpha4 * u_k[-2, j]
            + h[0] * phi_4_y_t(a, b, mu, j * h[1], k * tau)) / (beta4 * h[0] + alpha4)

In [ ]:
from math import cos, sin


def f_x_y_t(a: float, b: float, mu: float,
        x: float, y: float, t: float) -> float:
    return sin(x) * sin(y) * (MU * cos(MU * t) + (a + b) * sin(MU * t))


def phi_1_x_t(a: float, b: float, mu: float,
        x: float, t: float) -> float:
    return 0.0


def phi_2_x_t(a: float, b: float, mu: float,
        x: float, t: float) -> float:
    return -sin(x) * sin(mu * t)


def phi_3_y_t(a: float, b: float, mu: float,
        y: float, t: float) -> float:
    return 0.0


def phi_4_y_t(a: float, b: float, mu: float,
        y: float, t: float) -> float:
    return -sin(y) * sin(mu * t)

                
def psi_x_y(a: float, b: float, mu: float,
        x: float, y: float) -> float:
    return 0.0


def u_exact(a: float, b: float, mu: float,
        x: float, y: float, t: float) -> float:
    return sin(x) * sin(y) * sin(mu * t)

In [ ]:
from numpy import array


ALPHA1, BETA_1 = 0.0, 1.0
ALPHA_2, BETA_2 = 1.0, 0.0
ALPHA_3, BETA_3 = 0.0, 1.0
ALPHA_4, BETA_4 = 1.0, 0.0

n, k, ALPHA, MU, t = (128, 128), 1024, array([1.0, 1.0]), 1.0, 0.125